# SW Preparation
### ADB tool
* [Download the ADB zip file for Windows](https://dl.google.com/android/repository/platform-tools-latest-windows.zip)
* Extract the contents of this ZIP file into an easily accessible folder (such as C:\android\platform-tools)
* Connect your android phone / tablet and open command prompt to test if adb is working, detail commands can refer this [article](https://www.howtogeek.com/125769/how-to-install-and-use-abd-the-android-debug-bridge-utility/) 
* Add adb executable to Windows PATH [HowTo](https://www.howtogeek.com/118594/how-to-edit-your-system-path-for-easy-command-line-access/)

### OCR tool
* [Download tesseract Windows binary](https://github.com/tesseract-ocr/tesseract/wiki/Downloads)
* Install it as its installer instructed
* Open command prompt to verify the install
```
(base) C:\Users\rowe8002>tesseract --version
tesseract 3.05.02
 leptonica-1.75.3
  libgif 5.1.4 : libjpeg 8d (libjpeg-turbo 1.5.3) : libpng 1.6.34 : libtiff 4.0.9 : zlib 1.2.11 : libwebp 0.6.1 : libopenjp2 2.2.0
```
* Add tessract executable to Windows PATH [HowTo](https://www.howtogeek.com/118594/how-to-edit-your-system-path-for-easy-command-line-access/)

### Python package
* We recommand anacond3 for python package
    * run **conda search \[packagename\]** to find the package and **conda install \[packagename\]** to install it. 
    * In case the package doesn't in anaconda repository, run **pip search \[packagename\]** to find the package and **pip install \[packagename\]** to install it
* **PIL / threading / keyboard / os / pyaudio / subprocess / time / wave** are the packages required for this program

### Third party music recognition APP
* QQ music (this program was initally written for QQ music)

# HW Preparation
* android phone (or tablet)
* usb cable for PC and andriod device connection

Any question please contact wei.rong@nielsen.com

In [1]:
#!usr/bin/env python  
#coding=utf-8  

from PIL import Image
from threading import Thread
import keyboard
import os
import pyaudio
import subprocess
import time
import wave

In [2]:
########functions for wav playing########
def _wp_init(f):
    print("_wp_init()...")
    time.sleep(2)
    #instantiate PyAudio  
    p = pyaudio.PyAudio()  
    #open stream  
    stream = p.open(format = p.get_format_from_width(f.getsampwidth()),  
                channels = f.getnchannels(),  
                rate = f.getframerate(),  
                output = True)
    return p, stream

def _wp_run(f,c,s):
    print("_wp_run()...")
    data = f.readframes(c)
    while data:
        if keyboard.is_pressed('q'): break
        s.write(data)
        data = f.readframes(c)

def _wp_deInit(s,p,f):
    print("_wp_deInit()...")
    s.stop_stream()  
    s.close()
    f.close()
    p.terminate()

def wavPlaying(file):
    chunk = 1024
    wavfile = wave.open(file,"rb")  
    p, stream = _wp_init(wavfile)
    _wp_run(wavfile,chunk,stream)
    _wp_deInit(stream,p,wavfile)    

########functions for image crop########
def _im_crop_save(src,des,box):
    img = Image.open(src+".png")
    img2 = img.crop(box)
    img2.save(des+".png")

def _im_add_suffix(src,suffix):
    if os.path.isfile(src+'.png'):
        os.rename(src+'.png',src+'_'+suffix+'.png')
    
def _im_cleanup(name):
    for i in range(3):
        for j in ['png','txt']:
            f = name+'_check0'+str(i+1)+'.'+j
            try:
                os.remove(f)
            except OSError as e:
                #print(e)
                pass

def _im_cleanup_extracted_string(s):
    if s[:4] == '_ 一 ' and s[-4:] == ' 一 _': s = s[4:-4]
    if s[:2] == '一 ' and s[-2:] == ' 一': s = s[2:-2]
    if s[:2] == '— ' and s[-2:] == ' —': s = s[2:-2]
    if s[:1] == '_' and s[-1:] == '_': s = s[1:-1]
    return s.strip()
    
def _im_extract_track_artist(src):
    ret, error, track1, track2, artist = True, [], '', '', ''

    _im_crop_save(src, src+"_matchup_track", (120, 1255, 1080, 1365))
    
    track1 = _ocr_to_string(src+"_matchup_track")
    track1 = _im_cleanup_extracted_string(track1)
    if not len(track1): error.append('SCREEN_MATCHUP_EXTRACT_TRACK_NORMAL_FAIL')

    #try with psm7 as single line 
    track2 = _ocr_to_string(src+"_matchup_track",False,'-l chi_sim+eng -psm 7')
    track2 = _im_cleanup_extracted_string(track2)
    if not len(track2): error.append('SCREEN_MATCHUP_EXTRACT_TRACK_PSM7_FAIL')

    _im_crop_save(src, src+"_matchup_artist", (120, 1365, 1080, 1405))
    artist = _ocr_to_string(src+"_matchup_artist")
    artist = _im_cleanup_extracted_string(artist)
    if not len(artist): error.append('SCREEN_MATCHUP_EXTRACT_ARTIST_FAIL')

    if error == []: error.append('SCREEN_MATCHUP_EXTRACT_DONE')
    return ret, error, track1, track2, artist

########functions for image ocr########
def _ocr_to_string(img, cleanup=False, plus='-l chi_sim+eng'):
    subprocess.check_output('tesseract '+img+'.png'+' '+img+' '+plus, shell=True)
    text = ''
    with open(img + '.txt', 'r', encoding='UTF-8') as f:
        text = f.read().strip().replace('\n',' ')
    if cleanup:
        os.remove(img + '.txt')
    return text

########functions for android manipulation########
def _am_formatDuration(d):
    mins = d//60;
    secs = d - mins*60
    return "%02dm%02ds"%(mins,secs)
    
def _am_getDevices():
    cmd = "adb devices"
    out = subprocess.check_output(cmd.split())
    list = out.decode('utf-8').strip().split('\r\n')
    #print(list)
    return list

def _am_init():
    devices = _am_getDevices()
    if len(devices) <= 1 or '\tdevice' not in devices[-1]:
        cmd = "adb kill-server"
        out = subprocess.check_output(cmd.split())
        time.sleep(5)
        cmd = "adb start-server"
        out = subprocess.check_output(cmd.split())
        list = out.decode('utf-8').strip().split('\r\n')
        print(list)
        if 'successfully' not in list[-1]: return False
    return True

def _am_getScreenSize(st):
    print("%s:_am_getScreenSize " % _am_formatDuration(int(time.time()-st)),end='')
    cmd = "adb shell wm size"
    out = subprocess.check_output(cmd.split())
    size = tuple((out.decode('utf-8').strip().split('\r\n')[0].split(' ')[2].split('x')))
    print(size)
    return size

def _am_launchApp(st):
    time.sleep(2)
    print("%s:_am_launchApp" % _am_formatDuration(int(time.time()-st)))

def _am_CaptureFullScreen(des):
    cmd = "adb shell screencap -p /sdcard/screencap.png"
    out = subprocess.check_output(cmd.split())
    cmd = "adb pull /sdcard/screencap.png %s%s" % (des,".png")
    out = subprocess.check_output(cmd.split())
    if 'pulled' not in out.decode('utf-8').strip():
        print("%s:screencap or pull failed, exit..." % _am_formatDuration(int(time.time()-st)))
        return False, "SCREENCAP_OR_PULL_FAILED"
    return True, "SCREENCAP_AND_PULL_DONE"

def _am_pressBackThenRec():
    cmd = "adb shell input tap 40 90"           #this is back arrow position
    out = subprocess.check_output(cmd.split())
    cmd = "adb shell input tap 300 1715"        #this is recognition icon position
    out = subprocess.check_output(cmd.split())

def _am_pressRetry():
    cmd = "adb shell input tap 600 1530"        #this is re-try buttom
    out = subprocess.check_output(cmd.split())
    cmd = "adb shell input tap 300 1715"        #this is recognition icon position
    out = subprocess.check_output(cmd.split())
    
def _am_onScreenAction(fn,ss,st):
    print("%s:_am_onScreenAction " % _am_formatDuration(int(time.time()-st)), end='')
    
    ret, err = True, 'SCREEN_UNKNOWN'
    
    outdir = "_out"
    os.makedirs(outdir, exist_ok=True)

    #caputre the full sceen
    t = _am_formatDuration(int(time.time()-st))
    name = os.path.join(outdir,"%s_%s" % (fn,t))
    ret, err = _am_CaptureFullScreen(name)
    if not ret: return ret,err

    #crop to have multiple check-point sub-images to identify screen
    _im_crop_save(name, name+"_check01", (0, 180, 1200, 240))   #匹配到以下结果
    _im_crop_save(name, name+"_check02", (0, 610, 1200, 675))   #请求超时/未匹配到结果/无网络连接
    _im_crop_save(name, name+"_check03", (0, 1505, 1200, 1555)) #重试/停止识别/开始识别/重新识别/查看识别历史
    #_im_crop_save(name, name+"_check04", (0, 1750, 1200, 1795)) #听歌识曲 哼唱识别

    if '匹配到以下结果' in _ocr_to_string(name+"_check01"):
        #print(" SCREEN_MATCH_UP")
        ret, err, track1, track2, artist = _im_extract_track_artist(name)
        print(err)
        print(" trackX: %s"%track1)
        print(" trackY: %s"%track2)
        print(" artist: %s"%artist)
        _im_add_suffix(name,'matchup')
        time.sleep(5)
        _am_pressBackThenRec()
    elif '未匹配到结果' in _ocr_to_string(name+"_check02"):
        print("SCREEN_NO_MATCH")
        ret, err =  True, 'SCREEN_NO_MATCH'
        _im_add_suffix(name,'nomatch')
        _am_pressBackThenRec()
    elif '请求超时' in _ocr_to_string(name+"_check02"):
        print("SCREEN_REQ_TIMEOUT")
        ret, err =  True, 'SCREEN_REQ_TIMEOUT'
        _im_add_suffix(name,'timeout')
        _am_pressRetry()
    elif '发生错误了' in _ocr_to_string(name+"_check02"):
        print("SCREEN_ERROR_OCCUR")
        ret, err =  True, 'SCREEN_ERROR_OCCUR'
        _im_add_suffix(name,'errOccur')
        _am_pressRetry()
    elif '停止识别' in _ocr_to_string(name+"_check03"):
        print("SCREEN_MATCHING")
        ret, err =  True, 'SCREEN_MATCHING'
        _im_add_suffix(name,'matching')
    else:
        print("SCREEN_UNKNOWN")
        ret, err = True, 'SCREEN_UNKNOWN'
        _im_add_suffix(name,'unkown')
    
    _im_cleanup(name)
    return ret, err

def androidManipulation(t,fn):
    startTime = time.time()
    if not _am_init():
        print("android manipulation exit due to adb init failure...")
        return

    screenSize = _am_getScreenSize(startTime)
    _am_launchApp(startTime)
#    print(t.ident)
    while t.isAlive():
        ret,errorInfo = _am_onScreenAction(fn,screenSize,startTime)
        if not ret:
            print(errorInfo)
            break
        time.sleep(5)

    print("wav playing ends, exit androidManipulation()")

########wav play & android manipulation threads########
def main(f):
    threadWp = Thread(target=wavPlaying, args=(f,))
    threadWp.start()

    threadAm = Thread(target=androidManipulation, args=(threadWp,os.path.splitext(os.path.basename(f))[0],))
    threadAm.start()

    threadWp.join()
    threadAm.join()

In [4]:
#wavFolder = 'wav_folder'
wavFolder = 'wav_folder_test'

fileList = os.listdir(wavFolder)
wavList = []
for i in fileList:
    ext = os.path.splitext(os.path.basename(i))[1]
    if '.wav' == ext:
        wavList.append(i)

c = 0
for i in sorted(wavList):
    c = c + 1
    print("\n====================")
    print("[%03d: %s]"%(c,i))
    main(os.path.join(wavFolder,i))


[001: ahB.wav]
_wp_init()...
00m00s:_am_getScreenSize ('1200', '1920')
_wp_run()...
_wp_deInit()...
00m04s:_am_launchApp
wav playing ends, exit androidManipulation()

[002: cnn-07_3110_20180717191101.wav_sub_969_1617.wav]
_wp_init()...
00m00s:_am_getScreenSize ('1200', '1920')
_wp_run()...
_wp_deInit()...
00m04s:_am_launchApp
wav playing ends, exit androidManipulation()
